In [1]:
import keras_bitcoin
import dataset

Using TensorFlow backend.


In [2]:
df = dataset.get_labeled_dataset()

In [4]:
texts = df["text"]
labels = df["label"]

In [6]:
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size = keras_bitcoin.get_train_test_data(texts, labels)
model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length, max_sentence_size)

C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
770/770 [==============================] - 1s 804us/step - loss: 0.7033 - acc: 0.5065
Epoch 2/100
770/770 [==============================] - 0s 456us/step - loss: 0.6888 - acc: 0.5312
Epoch 3/100
770/770 [==============================] - 0s 384us/step - loss: 0.6860 - acc: 0.5260
Epoch 4/100
770/770 [==============================] - 0s 391us/step - loss: 0.6625 - acc: 0.6039
Epoch 5/100
770/770 [==============================] - 0s 396us/step - loss: 0.6349 - acc: 0.8351
Epoch 6/100
770/770 [==============================] - 0s 403us/step - loss: 0.5988 - acc: 0.8065
Epoch 7/100
770/770 [==============================] - 0s 371us/step - loss: 0.5434 - acc: 0.8571
Epoch 8/100
770/770 [==============================] - 0s 370us/step - loss: 0.4902 - acc: 0.9273
Epoch 9/100
770/770 [==============================] - 0s 378us/step - loss: 0.4351 - acc: 0.9532
Epoch 10/100
770/770 [==============================] - 0s 375us/step - loss: 0.3857 - acc: 0.9506
Epoch 11/100
770/77

770/770 [==============================] - 0s 382us/step - loss: 0.0401 - acc: 0.9831
Epoch 84/100
770/770 [==============================] - 0s 387us/step - loss: 0.0395 - acc: 0.9831
Epoch 85/100
770/770 [==============================] - 0s 370us/step - loss: 0.0395 - acc: 0.9831
Epoch 86/100
770/770 [==============================] - 0s 364us/step - loss: 0.0394 - acc: 0.9831
Epoch 87/100
770/770 [==============================] - 0s 361us/step - loss: 0.0395 - acc: 0.9831
Epoch 88/100
770/770 [==============================] - 0s 377us/step - loss: 0.0409 - acc: 0.9805
Epoch 89/100
770/770 [==============================] - 0s 382us/step - loss: 0.0394 - acc: 0.9818 0s - loss: 0.0347 - acc: 0.98
Epoch 90/100
770/770 [==============================] - 0s 381us/step - loss: 0.0395 - acc: 0.9831
Epoch 91/100
770/770 [==============================] - 0s 375us/step - loss: 0.0388 - acc: 0.9831
Epoch 92/100
770/770 [==============================] - 0s 368us/step - loss: 0.0387 - acc: 

### 4) Get Predictions

In [14]:
new_text = [
    "bitcoins sucks",
    "yip yip"
]

In [15]:
keras_bitcoin.get_predictions(new_text, model, vocab_length, max_sentence_size)

{'bitcoins sucks': 0.0}

### 5) Model Evaluation

In [7]:
loss, accuracy = model.evaluate(texts_test, labels_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

193/193 [==============================] - 0s 404us/step
Accuracy: 69.948184
